In [23]:
import numpy as np
from matplotlib import pyplot as plt,gridspec as gs
from afbf import tbfield,perfunction,coordinates,sdata
import os

In [24]:
# Taille de l'image
N=256
# Coordonnees pour synthetiser les image
coord=coordinates(N)
# Echantillonnage uniforme de l'espace
lags=coordinates()
lags.DefineUniformGrid(N,step=1,signed=True)

In [25]:
# Données pour estimer l'orientation
pi=np.pi
H_set_o=np.r_[0.01,0.25,0.35,0.45,0.5,0.65,0.75,0.85,0.99]
delta_set_o=np.r_[pi/64,9*pi/128,pi/8,3*pi/16,pi/4,5*pi/16,3*pi/8,7*pi/16,pi/2]
alpha_set_o=np.r_[-pi/2,-pi/6,0,pi/4,pi/3]

# Données pour estimer H et delta
theta1=pi/2
H_set=np.r_[0.01,0.15,0.25,0.35,0.45,0.5,0.65,0.75,0.85,0.99]
delta_set=np.r_[pi/64,9*pi/128,pi/8,3*pi/16,pi/4,5*pi/16,3*pi/8,7*pi/16,pi/2]

In [43]:
def nb_realstof_each(H_set,delta_set,alpha_set=None,nb_realst=1,file_dir='carrent_env',save=False):
    '''
    Entrée:\n'
    H_set: Tableau d'indices de Hurst.\n
    delta_set: Tableau de demi-ouvertures du cone fréquentiel.\n
    alpha: Tableau d'orientations du cone fréquentiel.\n
    nb_realst: Nombre de réalisations d'un champ de paramètre (H,delta,alpha)
    file_dir: Chemin de sauvgarde
    save: Bouléenne. Si True les données sont sauvegardées.
    Sortie:\n'
    nb_réalst réalisation(s) de chaque champ élémentaire.
    '''
    if (alpha_set==None).all():
        HDA=[];n=1
        for h in H_set:
            for d in delta_set:
                HDA.append([h,d])
    else:
        HDA=[];n=2
        for h in H_set:
            for d in delta_set:
                for a in alpha_set:
                    HDA.append([h,d,a])
    j=0
    np.random.seed(1)
    for hda in HDA:
        nb=0
        while nb<nb_realst:
            name='H='+str(hda[0])+'_delta='+str(hda[1])+'_alpha='+str(hda[n])+'_nb='+str(nb)+'.npy'
            if os.path.exists(file_dir+name)==False:
                np.random.seed(1)
                # Definition du champ
                Z=tbfield('efbf')
                # Modification des parametres du champ
                Z.hurst.ChangeParameters(fparam=np.r_[hda[0]])
                Z.topo.ChangeParameters(fparam=np.r_[0,1],finter=np.r_[-hda[1],hda[1]])
                Z.topo.ApplyTransforms(translate=-hda[n])
                Z.ComputeFeatures()     
                # Simulation
                z=Z.Simulate(coord)
                # Sauvegarde du champ
                if save!=None:
                    np.save(file_dir+'H='+str(hda[0])+'_delta='+str(hda[1])+'_alpha='+str(hda[n])+'_nb='+str(nb),z.values.reshape(z.M),)
            else:
                print(...)
            nb+=1
        j+=1
        print(j)